In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -r /content/drive/MyDrive/TG/tg2_COVIDNet/requirements.txt

In [1]:
train_ds_path = "/home/beduinigor/target_dataset/train"
test_ds_path = "/home/beduinigor/target_dataset/test"


In [2]:
batch_size = 32
image_size = (240, 240)
epochs = 500

from tensorflow.keras.preprocessing import image_dataset_from_directory

test_ds = image_dataset_from_directory(
    test_ds_path,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=False,
)

#Junta todos os batches
test_ds = test_ds.unbatch()


# TF tensors -> numpy array
test_ds = test_ds.as_numpy_iterator()

In [3]:
encoding = {"COVID-19": 2, "Normal": 0, "Pneumonia": 1}

In [ ]:
import cv2
import os
import numpy as np

img_array_train = []
target_array_train = []
img_array_test = []
target_array_test = []

for c in encoding:
    files = list(f for f in os.listdir(os.path.join(train_ds_path, c)) if os.path.isfile(os.path.join(train_ds_path, c, f)))
    for f in files:
        img = cv2.imread(os.path.join(train_ds_path, c, f), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, image_size)
        img = img/255.
        img = np.expand_dims(img, axis=2)
        img_array_train.append(img)
        target_array_train.append(encoding[c])
    files = list(f for f in os.listdir(os.path.join(test_ds_path, c)) if os.path.isfile(os.path.join(test_ds_path, c, f)))
    for f in files:
        img = cv2.imread(os.path.join(test_ds_path, c, f), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, image_size)
        img = img/255.
        img = np.expand_dims(img, axis=2)
        img_array_test.append(img)
        target_array_test.append(encoding[c])


In [ ]:
from random import shuffle

data_train = list(zip(img_array_train, target_array_train))
shuffle(data_train)
X_train, y_train = zip(*data_train)

data_test = list(zip(img_array_test, target_array_test))
shuffle(data_test)
X_test, y_test = zip(*data_test)

In [ ]:
import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
import os
from numpy import save

# ds_arrays_path = '/content/drive/MyDrive/TG/dataset_arraysTRAINING'
ds_arrays_path = '/home/beduinigor/dataset_arraysTRAINING'

if not os.path.isdir(ds_arrays_path):
    os.mkdir(ds_arrays_path)

save(os.path.join(ds_arrays_path, 'x_train_data.npy'), X_train)
save(os.path.join(ds_arrays_path, 'y_train_data.npy'), y_train)
save(os.path.join(ds_arrays_path, 'x_test_data.npy'), X_test)
save(os.path.join(ds_arrays_path, 'y_test_data.npy'), y_test)

In [4]:
import os
from numpy import load

# ds_arrays_path = '/content/drive/MyDrive/TG/dataset_arraysTRAINING'
ds_arrays_path = '/home/beduinigor/dataset_arraysTRAINING'

X_train = load(os.path.join(ds_arrays_path,'x_train_data.npy'))
y_train = load(os.path.join(ds_arrays_path,'y_train_data.npy'))
X_test = load(os.path.join(ds_arrays_path,'x_test_data.npy'))
y_test = load(os.path.join(ds_arrays_path,'y_test_data.npy'))

In [5]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import autokeras as ak
import numpy as np
from pathlib import Path
import os

optimizers = ['sgd', 'Adam']
for opt in optimizers:
    training_path = f"/home/beduinigor/training/{opt}"
    model_path = "/home/beduinigor/tg2_COVIDNet/models/final_model.h5"

    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor="loss",
        min_delta=0,
        patience=20,
        verbose=1,
        mode="auto",
        baseline=None,
        restore_best_weights=True)

    raw_training_path = os.path.join(training_path, "raw")
    augm_training_path = os.path.join(training_path, "augmented")

    for train_method_path in [raw_training_path, augm_training_path]:
        for subdir in ['models', 'tensorboard']:
            if not os.path.isdir(os.path.join(train_method_path, subdir)):
                path = Path(os.path.join(train_method_path, subdir))
                path.mkdir(parents=True, exist_ok=True)
    
    print("=============== AUGMENTED TRAINING ===============")
    # augmented training
    tensorboard_training_path = os.path.join(augm_training_path, 'tensorboard')
    models_training_path = os.path.join(augm_training_path, 'models')
    for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        n_fold = i + 1
        print(f"\n\n\nFOLD {n_fold}")
        print("-------------------------------")

        model = load_model(model_path, custom_objects=ak.CUSTOM_OBJECTS)
        config = model.get_config()
        clean_model = Model.from_config(config)
        clean_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

        tb_fold_path = os.path.join(tensorboard_training_path, f"fold_{n_fold}")
        fold_path = os.path.join(models_training_path, f"fold_{n_fold}")
        if not os.path.isdir(fold_path):
            os.mkdir(fold_path)
        if not os.path.isdir(tb_fold_path):
            os.mkdir(tb_fold_path)


        x_train_fold = X_train[train_index]
        y_train_fold = to_categorical(y_train[train_index], 3)

        x_test_fold = X_train[test_index]
        y_test_fold = to_categorical(y_train[test_index], 3)

        print("Saving training fold nparrays...")
        np.save(os.path.join(fold_path, f"x_train_arr_augm_fold_{n_fold}.npy"), x_train_fold)
        np.save(os.path.join(fold_path, f"y_train_arr_augm_fold_{n_fold}.npy"), y_train_fold)
        np.save(os.path.join(fold_path, f"x_test_arr_augm_fold_{n_fold}.npy"), x_test_fold)
        np.save(os.path.join(fold_path, f"y_test_arr_augm_fold_{n_fold}.npy"), y_test_fold)
        print("Arrays saved with sucess!")


        print("\nStart training...")  
        print("Class weights:")
        class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y_train[train_index]), y=y_train[train_index])
        class_weight = {i: class_weight[i] for i in range(len(class_weight))}
        print(class_weight)


        tb_callback = tf.keras.callbacks.TensorBoard(
            log_dir=tb_fold_path,
            histogram_freq=0,
            write_graph=True,
            write_images=False,
            update_freq="epoch",
            profile_batch=2,
            embeddings_freq=0,
            embeddings_metadata=None)

        callbacks = [early_stop_callback, tb_callback]
        
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
        
        clean_model.fit(datagen.flow(x_train_fold, y_train_fold, batch_size=batch_size),
                  epochs=epochs,
                  validation_data=(x_test_fold, y_test_fold),
                  callbacks=callbacks,
                  class_weight=class_weight)

        print("\nSaving model...")
        try:
            clean_model.save(os.path.join(fold_path, f"model_fold_{n_fold}.h5"), save_format='h5')
            print("Model saved with sucess!")
        except:
            print("Fail trying to save the model!")
    
    # raw training
    print("=============== RAW TRAINING ===============")
    tensorboard_training_path = os.path.join(raw_training_path, 'tensorboard')
    models_training_path = os.path.join(raw_training_path, 'models')
    for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        n_fold = i + 1
        print(f"\n\n\nFOLD {n_fold}")
        print("-------------------------------")

        model = load_model(model_path, custom_objects=ak.CUSTOM_OBJECTS)
        config = model.get_config()
        clean_model = Model.from_config(config)
        clean_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

        tb_fold_path = os.path.join(tensorboard_training_path, f"fold_{n_fold}")
        fold_path = os.path.join(models_training_path, f"fold_{n_fold}")
        if not os.path.isdir(fold_path):
            os.mkdir(fold_path)
        if not os.path.isdir(tb_fold_path):
            os.mkdir(tb_fold_path)


        x_train_fold = X_train[train_index]
        y_train_fold = to_categorical(y_train[train_index], 3)

        x_test_fold = X_train[test_index]
        y_test_fold = to_categorical(y_train[test_index], 3)

        print("Saving training fold nparrays...")
        np.save(os.path.join(fold_path, f"x_train_arr_raw_fold_{n_fold}.npy"), x_train_fold)
        np.save(os.path.join(fold_path, f"y_train_arr_raw_fold_{n_fold}.npy"), y_train_fold)
        np.save(os.path.join(fold_path, f"x_test_arr_raw_fold_{n_fold}.npy"), x_test_fold)
        np.save(os.path.join(fold_path, f"y_test_arr_raw_fold_{n_fold}.npy"), y_test_fold)
        print("Arrays saved with sucess!")


        print("\nStart training...")  
        print("Class weights:")
        class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y_train[train_index]), y=y_train[train_index])
        class_weight = {i: class_weight[i] for i in range(len(class_weight))}
        print(class_weight)


        tb_callback = tf.keras.callbacks.TensorBoard(
            log_dir=tb_fold_path,
            histogram_freq=0,
            write_graph=True,
            write_images=False,
            update_freq="epoch",
            profile_batch=2,
            embeddings_freq=0,
            embeddings_metadata=None)

        callbacks = [early_stop_callback, tb_callback]

        clean_model.fit(x=x_train_fold,
                  y=y_train_fold,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test_fold, y_test_fold),
                  callbacks=callbacks,
                  class_weight=class_weight)

        print("\nSaving model...")
        try:
            clean_model.save(os.path.join(fold_path, f"model_fold_{n_fold}.h5"), save_format='h5')
            print("Model saved with sucess!")
        except:
            print("Fail trying to save the model!")



=============== AUGMENTED TRAINING ===============



FOLD 1
-------------------------------
Saving training fold nparrays...
Arrays saved with sucess!

Start training...
Class weights:
{0: 0.5881866997108632, 1: 0.8487015751383568, 2: 8.224422442244224}
Epoch 1/500
  1/312 [..............................] - ETA: 0s - loss: 1.3542 - accuracy: 0.3125WARNING:tensorflow:From /home/beduinigor/covidnet_igor/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
312/312 [==============================] - 59s 189ms/step - loss: 1.0587 - accuracy: 0.5269 - val_loss: 0.9796 - val_accuracy: 0.5975
Epoch 2/500
312/312 [==============================] - 57s 181ms/step - loss: 0.8993 - accuracy: 0.6467 - val_loss: 0.7136 - val_accuracy: 0.7416
Epoch 3/500
312/312 [==============================] - 56s 179ms/step -

Epoch 51/500
312/312 [==============================] - 60s 192ms/step - loss: 0.1665 - accuracy: 0.9218 - val_loss: 0.5804 - val_accuracy: 0.8624
Epoch 52/500
312/312 [==============================] - 60s 192ms/step - loss: 0.1514 - accuracy: 0.9303 - val_loss: 0.4212 - val_accuracy: 0.8648
Epoch 53/500
312/312 [==============================] - 60s 192ms/step - loss: 0.1625 - accuracy: 0.9250 - val_loss: 0.4158 - val_accuracy: 0.8736
Epoch 54/500
312/312 [==============================] - 60s 191ms/step - loss: 0.1612 - accuracy: 0.9290 - val_loss: 0.6897 - val_accuracy: 0.7428
Epoch 55/500
312/312 [==============================] - 60s 191ms/step - loss: 0.1533 - accuracy: 0.9278 - val_loss: 1.2407 - val_accuracy: 0.7303
Epoch 56/500
312/312 [==============================] - 60s 191ms/step - loss: 0.1528 - accuracy: 0.9324 - val_loss: 0.9786 - val_accuracy: 0.7199
Epoch 57/500
312/312 [==============================] - 60s 191ms/step - loss: 0.1459 - accuracy: 0.9343 - val_loss: 1

Epoch 107/500
312/312 [==============================] - 60s 193ms/step - loss: 0.0355 - accuracy: 0.9846 - val_loss: 0.4949 - val_accuracy: 0.8724
Epoch 108/500
312/312 [==============================] - 60s 192ms/step - loss: 0.0290 - accuracy: 0.9868 - val_loss: 0.6352 - val_accuracy: 0.8327
Epoch 109/500
312/312 [==============================] - 60s 192ms/step - loss: 0.0758 - accuracy: 0.9681 - val_loss: 0.8508 - val_accuracy: 0.7905
Epoch 110/500
312/312 [==============================] - 60s 193ms/step - loss: 0.0497 - accuracy: 0.9773 - val_loss: 0.5449 - val_accuracy: 0.8836
Epoch 111/500
312/312 [==============================] - 60s 192ms/step - loss: 0.0322 - accuracy: 0.9848 - val_loss: 0.7251 - val_accuracy: 0.8266
Epoch 112/500
312/312 [==============================] - 60s 194ms/step - loss: 0.0352 - accuracy: 0.9837 - val_loss: 0.8992 - val_accuracy: 0.8519
Epoch 113/500
312/312 [==============================] - 60s 192ms/step - loss: 0.0343 - accuracy: 0.9853 - val_

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

from tensorflow.keras.models import load_model
import numpy as np
import os

ds_arrays_path = '/home/beduinigor/dataset_arraysTRAINING'
models_path = "/home/beduinigor/trainingADAM/models/kfold"

for root, subdirs, files in os.walk(models_path):
    for dir in subdirs:
        fold_path = os.path.join(root, dir)
        for f in os.listdir(fold_path):
            model_path = os.path.join(fold_path, f)
            if "model" in f:
                model = load_model(model_path)

                X_test = load(os.path.join(ds_arrays_path,'x_test_data.npy'))
                y_test = load(os.path.join(ds_arrays_path,'y_test_data.npy'))

                predictions = model.predict(x=X_test)
                num_predictions = np.argmax(predictions, axis=-1)
                
                class_report = classification_report(y_test, num_predictions, output_dict=True, target_names=["Normal", "Pneumonia", "COVID-19"])
                
                print(class_report['accuracy'])
                
                # print(confusion_matrix(y_test, num_predictions))
                # print(average_precision_score(y_test, num_predictions))
                # print(roc_auc_score(y_test, predictions, average='macro', multi_class='ovr'))
        